### 1) Prepare for AI Solution Registration
- [x] **등록 할 AI Contents 의 experimental_plan.yaml 를 ./config/ 에 준비해 둡니다.**

- [x] **가상 환경을 만들어 두고, ipykernel 을 제작해 둡니다.**     
<div style="margin-left: 20px">
```console
## ALO 의 main.py 위치에서 실행
conda create -n {name} python=3.10
conda init bash
conda activate {name}

python main.py 

pip install ipykernel
pip install requests 
python -m ipykernel install --user --name {name} --display-name [ipykernel-name]
```

- [x] **해당 jupyter notebook 에서 생성된 ipykernel 을 선택 합니다.**        
<div style="margin: 40px">
<img src="./image/ipykernel.png" width="300">
</div>

## 2) Confirm the operating status of AI Contents 
**_Sample Data 가 준비 되었는지 확인_**    
**_Asset 들이 준비 되었는지 확인_**     
**_Train & Inference 가 실행되는지 확인_**     
**_Train / Inference Artifacts 생성을 확인_**  

In [ ]:
import os
import json
import requests

from sm_control import SMC

## alo 로컬 실행
!python ../../main.py

<mark style="background: #FF5582A6;">TODO:</mark> pipeline.log 를 확인하는 방법 정리해 두기 

In [ ]:
name="bolt_fastening_table_classification_11_15"

In [ ]:
URI = 'http://10.158.2.243:9999/'

In [ ]:
t = requests.get(URI + 'api/v1/workspaces')


In [ ]:
# ECR, S3 정보 획득
# api로 치환
s3_bucket = t.json()[0]['s3_bucket_name']
ecr_uri = t.json()[0]['ecr_base_path']
repo = name
ws_name = t.json()[0]['name']


In [ ]:
# 저장되는 솔메 패스를 입력
SM_path = "./solution_meta_raw.yaml"

In [ ]:
sm = SMC(s3_bucket, ecr_uri)

In [ ]:
sm.set_yaml()

In [ ]:
sm.set_sm_description(name, "testing", f"s3://{s3_bucket}", f"s3://{s3_bucket}", "params", "alo", "s3://icon")

In [ ]:
# s3 접근확인
sm.s3_access_check()

In [ ]:
pipeline = "train"

In [ ]:
# s3 데이터 업로드
# 이전에 있던 데이터는 지워집니다
sm.s3_upload(pipeline)

In [ ]:
sm.set_container_uri(pipeline) # uri도 그냥 입력되게 수정


In [ ]:
sm.set_cadidate_param(pipeline)

In [ ]:
sm.set_artifacts_uri(pipeline)
sm.set_resource(pipeline)

In [ ]:
pipeline = "inference"

In [ ]:
# s3 데이터 업로드
# 이전에 있던 데이터는 지워집니다
sm.s3_upload(pipeline)

In [ ]:
sm.set_container_uri(pipeline) # uri도 그냥 입력되게 수정


In [ ]:
sm.set_cadidate_param(pipeline)

In [ ]:
sm.set_artifacts_uri(pipeline)
sm.set_resource(pipeline)
sm.set_model_uri(pipeline)

In [ ]:
sm.set_wrangler()

In [ ]:
sm.set_edge()

In [ ]:
# 등록을 위한 형태 변경
data = {
  "metadata_json": sm.sm_yaml
}

In [ ]:
#json으로 변경
data =json.dumps(data)

In [ ]:
#등록
post_response = requests.post('http://10.158.2.243:9999/api/v1/rass', data=data)
post_response_json = post_response.json()
post_response_json

In [ ]:
%store name

In [ ]:
rass_id = post_response_json['id']
%store rass_id